# Arxiv Map with Spacy

# Metodo 1 (lento - rischio ban)

In [1]:
#Libraries for web scraping
import requests
import urllib.request
from bs4 import BeautifulSoup
import time
#Libraries to manage our data
import pandas as pd

In [2]:
def retrieve_article(article_link):
    link_response = requests.get(article_link)
    time.sleep(1)
    soup = BeautifulSoup(link_response.text, "html.parser")
    title = soup.find(class_="title mathjax").text[6:]
    authors = soup.find(class_ = "authors").text[8:].split(', ')
    abstract = soup.find(class_="abstract mathjax").text[9:]
    return title, authors, abstract

In [3]:
def scrape_archive(subject, year, month):
    print(year)
    url2 = arxiv_site+"/"+"list/" + subject + "/" + year + month
    response = requests.get(url2)
    print(response.status_code == 200)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text,"html.parser")
        num_abs = int(soup.find("small").get_text()[10:16])
        print("Status code 200. Numero di articoli di "+str(month)+"/20"+str(year)+": ",num_abs)
        
        ### Codice funzionante. Da attivare nella versione definitiva
        
        #recuperiamo i link degli articoli
        links = []
        for i in range(1,int(num_abs/2000) + 1):
            new_url = url2 + "?skip=" + str((i-1)*2000) + "&show=" + str(2000)
            response = requests.get(new_url)
            soup = BeautifulSoup(response.text,"html.parser")
            for link in soup.find_all(title = "Abstract"):
                links.append([arxiv_site,link.get("href")])
        
        #recuperiamo i dati che inerenti i singoli articoli
        
        
        archive = []
        count_exc = 0
        count_abs = 1
        print(links)
        for article in links:
            try: 
                system_tuple = (2000 + int(year), int(month), subject,article[1])
                article_tuple = retrieve_article(article[0]+article[1])
                archive.append(article_tuple + system_tuple) 
                print(count_abs)
                count_abs+=1
            except:
                count_exc+=1
        print("Found ",count_exc,"exception found on ",num_abs," articles.")
        return archive
    else :
        print("Error: Status_code != 200")
        return 

In [4]:
#Generalized parameters
arxiv_site = "https://arxiv.org"

#codici anni presenti su Arxiv
years = ["19", "18", "17", "16", "15" , 
         "14", "13", "12", "11", "10", 
         "09", "08", "07", "06", "05",
         "04", "03", "02", "01", "00",
         "99", "98", "97", "96", "95",
         "94","93","92"]
#codici subjects presenti su Arxiv
subjects = ["math", "econ", "eess", "stat","q-fin","q-bio","CoRR",
            "astro-ph","cond-mat","gr-qc","hep-ex","hep-lat",
            "hep-ph","hep-th","math-ph","nlin","nucl-ex","nucl-th",
           "physics","quant-ph"]

#codici mesi 
months= ["01","02","03","04","05","06",
        "07","08","09","10","11","12"]

#labels del dataset
labels = ["Title","Authors","Abstract","system_year","system_month","system_subject","system_id"]

In [5]:
year = "18" #=2018
subject = "math"
month = "01"
url = arxiv_site+"/"+"list/"+subject+"/"+year+month

In [ ]:
df = pd.DataFrame(columns=labels)
#to do: loop over month and year
archive = scrape_archive(subject,year,month)
df.append(pd.DataFrame(archive,columns=labels))

18
True
Status code 200. Numero di articoli di 01/2018:  3353
[['https://arxiv.org', '/abs/1801.00013'], ['https://arxiv.org', '/abs/1801.00020'], ['https://arxiv.org', '/abs/1801.00023'], ['https://arxiv.org', '/abs/1801.00024'], ['https://arxiv.org', '/abs/1801.00028'], ['https://arxiv.org', '/abs/1801.00029'], ['https://arxiv.org', '/abs/1801.00032'], ['https://arxiv.org', '/abs/1801.00033'], ['https://arxiv.org', '/abs/1801.00034'], ['https://arxiv.org', '/abs/1801.00037'], ['https://arxiv.org', '/abs/1801.00038'], ['https://arxiv.org', '/abs/1801.00039'], ['https://arxiv.org', '/abs/1801.00043'], ['https://arxiv.org', '/abs/1801.00044'], ['https://arxiv.org', '/abs/1801.00045'], ['https://arxiv.org', '/abs/1801.00051'], ['https://arxiv.org', '/abs/1801.00053'], ['https://arxiv.org', '/abs/1801.00057'], ['https://arxiv.org', '/abs/1801.00058'], ['https://arxiv.org', '/abs/1801.00066'], ['https://arxiv.org', '/abs/1801.00068'], ['https://arxiv.org', '/abs/1801.00070'], ['https://arx

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# Metodo 2: Api Arxiv

Let's import the necessary libraries

In [54]:
import urllib
import feedparser
import time
import pandas as pd

In [55]:
# Base api query url
base_url = 'http://export.arxiv.org/api/query?';
# Search parameters
search_query = 'all:math.AP' # search for electron in all fields
start = 0                     # retreive the first 5 results
max_results = 25
# Opensearch metadata such as totalResults, startIndex, 
# and itemsPerPage live in the opensearch namespase.
# Some entry metadata lives in the arXiv namespace.
# This is a hack to expose both of these namespaces in
# feedparser v4.1
feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'
labels = ["system_id","title","authors","comment","categories","abstract"]

In [63]:
def retrive_all_abs(search_query, start, max_result):
    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)
    # perform a GET request using the base_url and query
    response = urllib.request.urlopen(base_url+query).read()
    # parse the response using feedparser
    feed = feedparser.parse(response)
    # print out feed information
    print('Feed title: %s' % feed.feed.title)
    print('Feed last updated: %s' % feed.feed.updated)
    # print opensearch metadata
    print('totalResults for this query: %s' % feed.feed.opensearch_totalresults)
    totalresults = 1000
    #feed.feed.opensearch_totalresults
    print('itemsPerPage for this query: %s' % feed.feed.opensearch_itemsperpage)
    itemsperpage = 100
    #feed.feed.opensearch_itemsperpage
    print('startIndex for this query: %s'   % feed.feed.opensearch_startindex)
    startindex = 0
    #feed.feed.opensearch_startindex
    
    archive  = []
    
    for i in range(int(startindex), int(totalresults), int(itemsperpage)):
        query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                             i,
                                                             int(itemsperpage))
        # perform a GET request using the base_url and query
        response = urllib.request.urlopen(base_url+query).read()
        # parse the response using feedparser
        feed = feedparser.parse(response)
        for entry in feed.entries:
            article = []
            article.append(entry.id.split('/abs/')[-1])
            #print('Published: %s' % entry.published)
            article.append(entry.title)
            
            # feedparser v5.0.1 correctly handles multiple authors, print them all
            try:
                all_authors= [author.name for author in entry.authors]
                #for author in entry.authors:
                #authors.append(all_authors)
                article.append(all_authors)
            except AttributeError:
                article.append([])
            
            try:
                comment = entry.arxiv_comment
                article.append(comment)
            except AttributeError:
                comment = 'No comment found'
                article.append(comment)
            
            # Lets get all the categories
            all_categories = [t['term'] for t in entry.tags]
            article.append(all_categories)
            
            # The abstract is in the <summary> element
            article.append(entry.summary)
            archive.append(article)
        time.sleep(4)
    return archive

In [64]:
abs_archive = pd.DataFrame(retrive_all_abs("math.AP",0,100),columns=labels)
df = pd.DataFrame(abs_archive,columns= labels)
df.info(), abs_archive

Feed title: ArXiv Query: search_query=math.AP&id_list=&start=0&max_results=25
Feed last updated: 2019-08-25T00:00:00-04:00
totalResults for this query: 37291
itemsPerPage for this query: 25
startIndex for this query: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 6 columns):
system_id     900 non-null object
title         900 non-null object
authors       900 non-null object
comment       900 non-null object
categories    900 non-null object
abstract      900 non-null object
dtypes: object(6)
memory usage: 42.3+ KB


(None,
           system_id                                              title  \
 0      1904.05029v1               Remarks on Lin-Nakamura-Wang's paper   
 1    math/0506402v2  Solutions for the quasi-linear equations in mu...   
 2    math/0001054v2  Statistical Mechanics of the Periodic Camassa-...   
 3    math/0006102v1  A Note on Closed Geodesics for a Class of non-...   
 4    math/0008078v2                   A Lax Pair for 2D Euler Equation   
 ..              ...                                                ...   
 895     1212.5926v1  An introduction to $BV$ functions in Wiener sp...   
 896     1212.5985v1  Boundary behavior of nonnegative solutions of ...   
 897     1212.6603v1  On solutions of quasilinear elliptic inequalit...   
 898     1212.6709v1  Blow up dynamics for equivariant critical Schr...   
 899     1301.0951v1  Soliton dynamics for the Schrodinger-Newton sy...   
 
                                     authors  \
 0                          [Masaru Ikehata

In [66]:
df.head(25)

,system_id,title,authors,comment,categories,abstract
0,1904.05029v1,Remarks on Lin-Nakamura-Wang's paper,[Masaru Ikehata],7 pages,"[math.AP, 35R30]",Theorem 1.2 in their paper arXiv:1904.00999v1 ...
1,math/0506402v2,Solutions for the quasi-linear equations in mu...,[Sadek Gala],This paper has been withdrawn,"[math.AP, 35D05, 35Dxx]",This paper has been removed by arXiv administr...
2,math/0001054v2,Statistical Mechanics of the Periodic Camassa-...,"[Adrian Constantin, Doron Levy]",0 pages,"[math.AP, Analysis of PDEs]",The paper has been withdrawn
3,math/0006102v1,A Note on Closed Geodesics for a Class of non-...,[Simone Secchi],11 pages,"[math.AP, math.DG, 58E30]",We prove the existence of multiple closed geod...
4,math/0008078v2,A Lax Pair for 2D Euler Equation,[Yanguang Charles Li],3 pages,"[math.AP, math-ph, math.MP, 35;51]",A Lax pair for the 2D Euler equation is found.
5,math/0011195v1,Multiplicity results for some nonlinear Schroe...,"[A. Ambrosetti, A. Malchiodi, S. Secchi]",No comment found,[math.AP],We prove some multiplicity results for a nonli...
6,math/0104139v2,On the biharmonic Dirichlet problem: The highe...,"[Atanas Stefanov, Gregory C. Verchota]",Withdrawn by authors,"[math.AP, math.CA, 35J40]",Withdrawn by authors.
7,math/0106075v2,Critical functions on Riemannian Manifold wich...,[Moubinool Omarjee],No comment found,"[math.AP, 58-XX]",We study the existence of extremal functions o...
8,math/0210142v2,Nonlinear differential equations on non-compac...,[Simone Secchi],"Ph.D. Thesis, 3 figures",[math.AP],We present some results in the analysis of non...
9,math/0212376v1,A Liouville type theorem for some conformally ...,"[Aobing Li, YanYan Li]",No comment found,"[math.AP, 35J60; 53C21; 58J05]",We establish a Liouville type theorem for some...


In [78]:
df.to_csv('arxiv_sample.csv',index = True)

In [74]:
pwd

'C:\\Users\\rdemaio\\Project_Arxiv_Spacy'

# Codice Test

In [13]:
#Chuck per test del progetto. Utile per portare solo una pagina (25 Abs) di link
response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")
links = []
for link in soup.find_all(title = "Abstract"):
    links.append([arxiv_site,link.get("href")])

In [15]:
#link di test
df = pd.DataFrame(columns=labels)
archive = []
for link  in links:
    system_tuple = (2000 + int(year), int(month), subject,link[1])
    archive.append(retrieve_article(link[0]+link[1]) + system_tuple) 
df.append(pd.DataFrame(archive,columns=labels))

,Title,Authors,Abstract,system_year,system_month,system_subject,system_id
0,Birational geometry of algebraic varieties,[Caucher Birkar],This is a report on some of the main develop...,2018,1,math,/abs/1801.00013
1,K-theory and actions on Euclidean retracts,[Arthur Bartels],This note surveys axiomatic results for the ...,2018,1,math,/abs/1801.00020
2,Exceptional sets for nonuniformly hyperbolic d...,"[Sara Campos, Katrin Gelfert]","For a surface diffeomorphism, a compact inva...",2018,1,math,/abs/1801.00023
3,On the topological equivalence of S-metric and...,[Nihal Taş],The aim of this paper is to establish the eq...,2018,1,math,/abs/1801.00024
4,Smooth quotients of abelian varieties by finit...,"[Robert Auffarth, Giancarlo Lucchini Arteche]",We give a complete classification of smooth ...,2018,1,math,/abs/1801.00028
5,Graded Expectations: Betti numbers and anti-le...,"[Alexander Engström, Christian Go, Matthew T. ...",This paper examines the one-to-one-to-one co...,2018,1,math,/abs/1801.00029
6,On uniqueness of end sums and 1-handles at inf...,"[Jack S. Calcut, Robert E. Gompf]",For oriented manifolds of dimension at least...,2018,1,math,/abs/1801.00032
7,Concerning $q$-summable Szlenk index,[Ryan M. Causey],For each ordinal $\xi$ and each $1\leqslant ...,2018,1,math,/abs/1801.00033
8,Mean field matching and TSP in pseudo-dimension 1,"[Giorgio Parisi, Johan Wästlund]",Recent work on optimization problems in rand...,2018,1,math,/abs/1801.00034
9,"On linear sections of the spinor tenfold, I",[Alexander Kuznetsov],We discuss the geometry of transverse linear...,2018,1,math,/abs/1801.00037


In [236]:
response = requests.get("https://arxiv.org/list/math/1908")
soup = BeautifulSoup(response.text,"html.parser")
num_abs = int(soup.find("small").get_text()[10:16])
soup.find("small").get_text()[10:16]

In [ ]:
df.head(10)

In [263]:
links = []
for i in range(1,int(3447/2000) + 1):
    new_url = "https://arxiv.org/list/math/1901" + "?skip=" + str((i-1)*2000) + "&show=" + str(2000)
    response = requests.get(new_url)
    soup = BeautifulSoup(response.text,"html.parser")
    for link in soup.find_all(title = "Abstract"):
        links.append(arxiv_site+link.get("href"))

In [266]:
links[-200:-1]

['https://arxiv.org/abs/1901.06599',
 'https://arxiv.org/abs/1901.06602',
 'https://arxiv.org/abs/1901.06605',
 'https://arxiv.org/abs/1901.06607',
 'https://arxiv.org/abs/1901.06612',
 'https://arxiv.org/abs/1901.06619',
 'https://arxiv.org/abs/1901.06621',
 'https://arxiv.org/abs/1901.06624',
 'https://arxiv.org/abs/1901.06629',
 'https://arxiv.org/abs/1901.06630',
 'https://arxiv.org/abs/1901.06632',
 'https://arxiv.org/abs/1901.06633',
 'https://arxiv.org/abs/1901.06634',
 'https://arxiv.org/abs/1901.06636',
 'https://arxiv.org/abs/1901.06639',
 'https://arxiv.org/abs/1901.06642',
 'https://arxiv.org/abs/1901.06644',
 'https://arxiv.org/abs/1901.06652',
 'https://arxiv.org/abs/1901.06660',
 'https://arxiv.org/abs/1901.06664',
 'https://arxiv.org/abs/1901.06665',
 'https://arxiv.org/abs/1901.06666',
 'https://arxiv.org/abs/1901.06668',
 'https://arxiv.org/abs/1901.06669',
 'https://arxiv.org/abs/1901.06670',
 'https://arxiv.org/abs/1901.06671',
 'https://arxiv.org/abs/1901.06673',
 